In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification, BertAdam, BertForNextSentencePrediction
from tqdm import tqdm, trange
import torch.nn as nn
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import sys
sys.path.append('../')
import Drain
import pickle 

from torch.nn import functional as F



In [2]:
from loglizer import *

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.set_device(1)

In [4]:
def parse_logs(input_dir, output_dir, log_format, st=0.3, depth=3, log_file='bgl_payload'): 
    regex = [] #empty, we don't have to use it.

    parser = Drain.LogParser(log_format, indir=input_dir, outdir=output_dir,  depth=depth, st=st, rex=regex)
    parser.parse(log_file)
    
#parsing the logs, extracting templates and parameters
input_dir  = '../data/'  # The input directory of log file
output_dir = '../output/'  # The output directory of parsing results
#Hostname,user_id,project_domain,Timestamp,@timestamp,log_level,Pid,Payload,tenant_id,programname,request_id,python_module,Logger,user_domain,domain_id,http_status,http_method,http_version,http_url
log_format = '<Content>' 
# parse_logs(input_dir, output_dir, log_format)

In [5]:
df = pd.read_csv('./datasets/data/BGL_log_structured_after_preprocessing.csv')
print(len(df))

4713493


In [6]:
from datetime import datetime
df.Time = df.Time.apply(lambda x: datetime.strptime(x, "%Y-%m-%d-%H.%M.%S.%f"))

In [75]:
S = pd.to_datetime(df.Time)
# print((S - S[0]).astype('timedelta64[6h]'))
L = [g.reset_index(drop=True) for i, g in df.groupby([(S - S[0]).astype('timedelta64[6h]')])]
df.index = S
# L = [g.reset_index(drop=True) for i, g in df.groupby(pd.Grouper(freq='6h', loffset=pd.timedelta_range(start=datetime.S[0], periods=0, freq="30min")))]


In [5]:
def BGL_targests():
    log_file = "datasets/data/" + 'BGL.log'
    pom_lista = []
    every_n = 1
    max_lines = 5000000 # for spirit is False

    with open(log_file, 'r', encoding="latin-1") as fin:
        for i, line in enumerate(fin):
            if i % every_n == 0:
                try:
                    pom_lista.append(line.rsplit(" ")[0])
                except Exception as e:
                    pass
            if max_lines:
                if i == max_lines:
                    break
    return np.where(np.array(pom_lista) != '-', 1, 0 )

In [77]:
true_labels = BGL_targests()

In [81]:

training_data = []
labels = []
tr_labels = []

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

encoded_data = le.fit_transform(df.EventId)

training_data = []
tr_labels = []

for i in trange(0, len(L)):
    training_data.append(encoded_data[L[i].iloc[:, 0].values])
    tr_labels.append(np.where(true_labels[L[i].iloc[:, 0].values].sum()>0, 1, 0).tolist())

100%|██████████| 828/828 [00:00<00:00, 7430.60it/s]


In [83]:
data_len = len(training_data)
len(training_data), len(tr_labels)

(828, 828)

In [85]:
training_data = np.array(training_data)
labels = np.array(tr_labels)

In [7]:
import pickle
from collections import defaultdict

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder


from loglizer.loglizer.models import DecisionTree
from loglizer.loglizer.models import IsolationForest
from loglizer.loglizer.models import PCA
from loglizer.loglizer.models import LR
from loglizer.loglizer.models import InvariantsMiner
from loglizer.loglizer import preprocessing

results_windows = defaultdict(dict)


for tw in ['5m', '15m', '30m', '60m', '180m', '360m', '540m', '720m']:
    time_window = tw
    print("The curent time window is {}".format(time_window))


    df = pd.read_csv('./datasets/data/BGL_log_structured_after_preprocessing.csv')
    print(len(df))


    from datetime import datetime
    df.Time = df.Time.apply(lambda x: datetime.strptime(x, "%Y-%m-%d-%H.%M.%S.%f"))


    S = pd.to_datetime(df.Time)
    # print((S - S[0]).astype('timedelta64[6h]'))
    L = [g.reset_index(drop=True) for i, g in df.groupby([(S - S[0]).astype('timedelta64['+time_window+']')])]
    df.index = S
    # L = [g.reset_index(drop=True) for i, g in df.groupby(pd.Grouper(freq='6h', loffset=pd.timedelta_range(start=datetime.S[0], periods=0, freq="30min")))]

    true_labels = BGL_targests()


    training_data = []
    labels = []
    tr_labels = []

    from sklearn.preprocessing import LabelEncoder

    le = LabelEncoder()

    encoded_data = le.fit_transform(df.EventId)

    training_data = []
    tr_labels = []

    for i in trange(0, len(L)):
        training_data.append(encoded_data[L[i].iloc[:, 0].values])
        tr_labels.append(np.where(true_labels[L[i].iloc[:, 0].values].sum()>0, 1, 0).tolist())


    data_len = len(training_data)
    len(training_data), len(tr_labels)    

    training_data = np.array(training_data)
    labels = np.array(tr_labels)    


    resutls_run_IF = defaultdict()
    resutls_run_PCA = defaultdict()
    resutls_run_DT = defaultdict()
    resutls_run_LR = defaultdict()
    resutls_run_IM = defaultdict()

    # with open("./log_anomaly_detection_evaluation/baselines/Counts_IsolationForest_results_BGL.pickle", "rb") as file:
    #     resutls_run_IF  = pickle.load(file)

    # with open("./log_anomaly_detection_evaluation/baselines/Counts_DecisionTree_results_BGL.pickle", "rb") as file:
    #     resutls_run_DT  = pickle.load(file)

    # with open("./log_anomaly_detection_evaluation/baselines/Counts_PCA_results_BGL.pickle", "rb") as file:
    #     resutls_run_PCA  = pickle.load(file)

    # with open("./log_anomaly_detection_evaluation/baselines/Counts_LR_results_BGL.pickle", "rb") as file:
    #     resutls_run_LR  = pickle.load(file)

    # with open("./log_anomaly_detection_evaluation/baselines/Counts_IM_results_BGL.pickle", "rb") as file:
    #     resutls_run_LR  = pickle.load(file)



    for ratio in [0.05, 0.1, 0.2, 0.4, 0.6, 0.8]:
    # for ratio in [0.8]:

        tr_x = training_data[:int(ratio*data_len)]
        te_x = training_data[int(ratio*data_len):]

        tr_labels = labels[:int(ratio*data_len)]
        ts_labels = labels[int(ratio*data_len):]

        feature_extractor = preprocessing.FeatureExtractor()
        x_train = feature_extractor.fit_transform(tr_x, term_weighting=True, normalization='zero-mean')
        x_train = feature_extractor.fit_transform(tr_x)


        model = IsolationForest(contamination="auto", verbose=5)
        model.fit(x_train)

        x_test = feature_extractor.transform(te_x)
        p1 = model.predict(x_test)
        p2 = ts_labels

        d = {}
        d["acc"] = accuracy_score(p2,p1)
        d["precision"] = precision_score(p2,p1)
        d["recall"] = recall_score(p2,p1)
        d["f1"] = f1_score(p2,p1)

        resutls_run_IF[ratio] = d
        with open("./log_anomaly_detection_evaluation/baselines/Counts_IsolationForest_results_BGL"+ str(time_window) +"_.pickle", "wb") as file:
            pickle.dump(resutls_run_IF, file)

        model = DecisionTree()
        model.fit(x_train, tr_labels)
        p1 = model.predict(x_test)
        p2 = ts_labels

        d = {}
        d["acc"] = accuracy_score(p2,p1)
        d["precision"] = precision_score(p2,p1)
        d["recall"] = recall_score(p2,p1)
        d["f1"] = f1_score(p2,p1)


        resutls_run_DT[ratio] = d
        with open("./log_anomaly_detection_evaluation/baselines/Counts_DecisionTree_results_BGL_"+ str(time_window) +"_.pickle", "wb") as file:
            pickle.dump(resutls_run_DT, file)



        model = PCA()
        model.fit(x_train)
        p1 = model.predict(x_test)
        p2 = ts_labels

        d = {}
        d["acc"] = accuracy_score(p2,p1)
        d["precision"] = precision_score(p2,p1)
        d["recall"] = recall_score(p2,p1)
        d["f1"] = f1_score(p2,p1)


        resutls_run_PCA[ratio] = d
        with open("./log_anomaly_detection_evaluation/baselines/Counts_PCA_results_BGL_"+ str(time_window) +"_.pickle", "wb") as file:
            pickle.dump(resutls_run_PCA, file)



        model = LR()
        model.fit(x_train, tr_labels)
        p1 = model.predict(x_test)
        p2 = ts_labels

        d = {}
        d["acc"] = accuracy_score(p2,p1)
        d["precision"] = precision_score(p2,p1)
        d["recall"] = recall_score(p2,p1)
        d["f1"] = f1_score(p2,p1)




        resutls_run_LR[ratio] = d
        with open("./log_anomaly_detection_evaluation/baselines/Counts_LR_results_BGL_"+ str(time_window) +"_.pickle", "wb") as file:
            pickle.dump(resutls_run_LR, file)


    #     model = InvariantsMiner()
    #     model.fit(x_train)
    #     p1 = model.predict(x_test)
    #     p2 = ts_labels

    #     d = {}
    #     d["acc"] = accuracy_score(p2,p1)
    #     d["precision"] = precision_score(p2,p1)
    #     d["recall"] = recall_score(p2,p1)
    #     d["f1"] = f1_score(p2,p1)



    #     resutls_run_IM[ratio] = d
    #     with open("./log_anomaly_detection_evaluation/baselines/Counts_IM_results_BGL.pickle", "wb") as file:
    #         pickle.dump(resutls_run_IM, file)


        f = open("results.txt", "a+")
        f.write("Evaluated until {}\n".format(ratio))
        f.close()

    d = {}
    d['LR'] = resutls_run_LR
    d['PCA'] = resutls_run_PCA
    d['IF'] = resutls_run_IF
    d['DT'] = resutls_run_DT

    results_windows[time_window]=d
    with open("./log_anomaly_detection_evaluation/baselines/results_different_window_size/count_bgl.pickle", "wb") as file:
        pickle.dump(results_windows, file)

The curent time window is 5m
4713493


100%|██████████| 14499/14499 [00:01<00:00, 9937.28it/s]
/opt/tljh/user/lib/python3.6/site-packages/sklearn/ensemble/_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  FutureWarning
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.


====== Transformed train data summary ======
Train data shape: 724-by-124

====== Transformed train data summary ======
Train data shape: 724-by-124

====== Model summary ======
Building estimator 1 of 17 for this parallel run (total 100)...Building estimator 1 of 17 for this parallel run (total 100)...
Building estimator 1 of 17 for this parallel run (total 100)...
Building estimator 1 of 17 for this parallel run (total 100)...
Building estimator 1 of 16 for this parallel run (total 100)...
Building estimator 1 of 16 for this parallel run (total 100)...

Building estimator 2 of 17 for this parallel run (total 100)...


[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    0.1s finished


Building estimator 2 of 17 for this parallel run (total 100)...
Building estimator 2 of 16 for this parallel run (total 100)...
Building estimator 2 of 16 for this parallel run (total 100)...Building estimator 2 of 17 for this parallel run (total 100)...Building estimator 2 of 17 for this parallel run (total 100)...


Building estimator 3 of 17 for this parallel run (total 100)...
Building estimator 3 of 17 for this parallel run (total 100)...
Building estimator 3 of 16 for this parallel run (total 100)...
Building estimator 3 of 17 for this parallel run (total 100)...Building estimator 3 of 17 for this parallel run (total 100)...

Building estimator 3 of 16 for this parallel run (total 100)...
Building estimator 4 of 16 for this parallel run (total 100)...Building estimator 4 of 17 for this parallel run (total 100)...Building estimator 4 of 17 for this parallel run (total 100)...


Building estimator 4 of 17 for this parallel run (total 100)...Building estimator 4 of 17 for this paral

...

/opt/tljh/user/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Train data shape: 340-by-338

====== Transformed train data summary ======
Train data shape: 340-by-338

====== Model summary ======
Building estimator 1 of 17 for this parallel run (total 100)...Building estimator 1 of 17 for this parallel run (total 100)...
Building estimator 1 of 17 for this parallel run (total 100)...

Building estimator 1 of 17 for this parallel run (total 100)...
Building estimator 1 of 16 for this parallel run (total 100)...
Building estimator 1 of 16 for this parallel run (total 100)...
Building estimator 2 of 16 for this parallel run (total 100)...
Building estimator 2 of 17 for this parallel run (total 100)...Building estimator 2 of 17 for this parallel run (total 100)...

Building estimator 2 of 17 for this parallel run (total 100)...
Building estimator 3 of 16 for this parallel run (total 100)...
Building estimator 3 of 17 for this parallel run (total 100)...
Building estimator 2 of 16 for this parallel run (total 100)...
Building estimator 3 of 17 for this

/opt/tljh/user/lib/python3.6/site-packages/sklearn/ensemble/_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  FutureWarning
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    0.1s finished


====== Transformed test data summary ======
Test data shape: 86-by-338

====== Model summary ======
====== Model summary ======
n_components: 5
Project matrix shape: 338-by-338
SPE threshold: 1043270.0138790032

====== Model summary ======


/opt/tljh/user/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [87]:
import pickle
with open("./log_anomaly_detection_evaluation/baselines/Counts_IsolationForest_results_BGL.pickle", "rb") as file:
    resutls_run_IF  = pickle.load(file)
    
with open("./log_anomaly_detection_evaluation/baselines/Counts_DecisionTree_results_BGL.pickle", "rb") as file:
    resutls_run_DT  = pickle.load(file)
    
with open("./log_anomaly_detection_evaluation/baselines/Counts_PCA_results_BGL.pickle", "rb") as file:
    resutls_run_PCA  = pickle.load(file)
    
with open("./log_anomaly_detection_evaluation/baselines/Counts_LR_results_BGL.pickle", "rb") as file:
    resutls_run_LR  = pickle.load(file)


In [ ]:
# model = DeepLog(hidden_size=128, output_size=len(template2index.keys()))
# model.load_state_dict(torch.load('model_n.pt'))
# model.to(device)
model.eval()


preds1 = []
l = []
# Tracking variables
test_loss = 0
nb_te_examples, nb_te_steps = 0, 0
# Train the data for one epoch
print(len(test_dataloader))
for step, batch in enumerate(test_dataloader):
    if step % 200 ==0:
        print("Step:", step)
        # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input, b_labels = batch
    # Forward pass
    with torch.no_grad():
        predictions = model(b_input)
       #print(predictions.cpu().numpy().argsort(axis=1)[:,-20:].shape)
        preds1  += list(predictions.cpu().numpy().argsort(axis=1).astype(np.int16))
        l += list(b_labels.cpu().numpy().astype(np.int16))
              #del predictions


In [ ]:
from collections import defaultdict



# results_deeplog = defaultdict(dict)


for t in [1, 3, 9, 20, 50, 100, 250, 500, 750, 1000, 1500]:
# for t in [ 750, , 1500]:
    res1 = []
    br = 0
    for (x, y) in zip(preds1, l):
        if br % 100000 == 0:
            print(br)
        br += 1

        if y in x[(-1)*t:]:
            res1.append(0)
        else:
            res1.append(1)


    #for anomalies, .. false negatives
    p1 = res1
    p2 = test_labels_ground


    from sklearn.metrics import accuracy_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import confusion_matrix

    print("The accuracy score is: {}".format(accuracy_score(p2,p1)))
    print("The precision score is: {}".format(precision_score(p2, p1)))
    print("The recall score is: {}".format(recall_score(p2, p1)))
    print("The f1 score is: {}".format(f1_score(p2,p1)))
        
    d = {}
    d["acc"] = accuracy_score(p2,p1)
    d["precision"] = precision_score(p2,p1)
    d["recall"] = recall_score(p2,p1)
    d["f1"] = f1_score(p2,p1)
    
    results_deeplog[t] = d
    resutls_run[ratio] = results_deeplog
    
    print(confusion_matrix(p2, p1))

In [ ]:
resutls_run

In [ ]:
a = predictions.cpu().numpy().argsort(axis=1)

In [ ]:
p2 = test_labels_ground
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix

print("The accuracy score is: {}".format(accuracy_score(p2,p1)))
print("The precision score is: {}".format(precision_score(p2, p1)))
print("The recall score is: {}".format(recall_score(p2, p1)))
print("The f1 score is: {}".format(f1_score(p2,p1)))
print(confusion_matrix(p2, p1))